# Ontological validation in `pyiron_workflow`

In `pyiron_workflow`, we leverage pyiron's ontological validation of workflow graphs found in `semantikon`. We strive to support every syntactic aspect of `semantikon`, which you can read more about on [the GitHub repository](https://github.com/pyiron/semantikon) or [the docs]() for that project.

This is accomplished practically by wrapping function IO (or dataclass nodes) with the `semantikon.metadata.u` call. Here, we demonstrate a variety of use-cases, and show how this functionality can be taken further to get data type- and ontologically-valid suggestions for new connections in your workflow, or new nodes to add to it.

Note that ontological typing _only_ works when there's a parent object around (a macro or a workflow).

# Ontological connection checking

Some success and failure cases in the presence or absence of ontological hints. A key takeway is that these hints function very much like `pyiron_workflow`'s regular type hinting: if hints are present on both sides of a new connection, they _must_ be valid, but if one or both sides are missing the hint we skip the validation.

In [28]:
import rdflib

from semantikon.metadata import u

import pyiron_workflow as pwf
from pyiron_workflow import suggest
from pyiron_workflow.channels import ChannelConnectionError
from pyiron_workflow.nodes.composite import FailedChildError


EX = rdflib.Namespace("http://www.example.org/")

class Meal: ...

class Garbage: ...

@pwf.as_function_node("pizza")
def prepare_pizza() -> u(Meal, uri=EX.Pizza):
    return Meal()

@pwf.as_function_node("unidentified_meal")
def prepare_non_ontological_meal() -> Meal:
    return Meal()

@pwf.as_function_node("rice")
def prepare_rice() -> u(Meal, uri=EX.Rice):
    return Meal()

@pwf.as_function_node("garbage")
def prepare_garbage() -> u(Garbage, uri=EX.Garbage):
    return Garbage()

@pwf.as_function_node("garbage")
def prepare_unhinted_garbage():
    return Garbage()

@pwf.as_function_node("verdict")
def eat(meal: u(Meal, uri=EX.Meal)) -> str:
    return f"Yummy {meal.__class__.__name__} meal"

@pwf.as_function_node("verdict")
def eat_pizza(meal: u(Meal, uri=EX.Pizza)) -> str:
    return f"Yummy {meal.__class__.__name__} pizza"

## Both fully hinted

Works fine

In [29]:
wf = pwf.Workflow("ontoflow")
wf.make = prepare_pizza()
wf.eat = eat_pizza(wf.make)
wf()

{'eat__verdict': 'Yummy Meal pizza'}

## Upstream type hint is missing

Standard `pyiron_workflow` typing behaviour: we are allowed to form the connection (since the source has no hint), but at runtime, we will fail when we try to actually assign the value

In [30]:
wf = pwf.Workflow("no_type")
wf.make = prepare_unhinted_garbage()
wf.eat = eat_pizza(wf.make)
try:
    wf.recovery = None
    wf()
except FailedChildError as e:
    print(e)

/no_type encountered error in child: {'/no_type/eat.accumulate_and_run': TypeError("The channel /no_type/eat.meal cannot take the value `<__main__.Garbage object at 0x132838950>` (<class '__main__.Garbage'>) because it is not compliant with the type hint typing.Annotated[__main__.Meal, ('uri', rdflib.term.URIRef('http://www.example.org/Pizza'))]")}


## Upstream type hint is wrong

Standard `pyiron_workflow` typing behaviour: we're not even allowed to form the connection -- the recipe would be invalid

In [31]:
wf = pwf.Workflow("no_type")
wf.make = prepare_garbage()
try:
    wf.eat = eat_pizza(wf.make)
except ChannelConnectionError as e:
    print(e)

The upstream channel /no_type/make.garbage cannot connect to the downstream channel /no_type/eat_pizza.meal because the upstream type hint (typing.Annotated[__main__.Garbage, ('uri', rdflib.term.URIRef('http://www.example.org/Garbage'))]) is not as or more specific than the downstream type hint (typing.Annotated[__main__.Meal, ('uri', rdflib.term.URIRef('http://www.example.org/Pizza'))]).


So far, so good: `u` decoration has no negative impact on the existing type hint checking procedures

## Upstream ontological hint is missing

New ontological behaviour: As with type hints, if one side is missing we just let things pass. Unlike type hints, we can also _execute_ the workflow, because the ontologies only impact the recipe-level behaviour, not the instance behaviour!

In [32]:
wf = pwf.Workflow("no_ontology")
wf.make = prepare_non_ontological_meal()
wf.eat = eat_pizza(wf.make)
wf()

{'eat__verdict': 'Yummy Meal pizza'}

## Upstream ontological hint is WRONG

New ontological behaviour: new ontological type checking now prevents us from even forming the ontologically invalid connection!

In [33]:
wf = pwf.Workflow("failed_ontology")
wf.make = prepare_rice()
try:
    wf.eat = eat_pizza(wf.make)
except ChannelConnectionError as e:
    print(e)

The upstream channel /failed_ontology/make.rice cannot connect to the downstream channel /failed_ontology/eat_pizza.meal because the upstream type hint (typing.Annotated[__main__.Meal, ('uri', rdflib.term.URIRef('http://www.example.org/Rice'))]) and downstream type hint (typing.Annotated[__main__.Meal, ('uri', rdflib.term.URIRef('http://www.example.org/Pizza'))]) produce a non-empty ontological validation report:
{'missing_triples': [], 'incompatible_connections': [(rdflib.term.URIRef('failed_ontology.eat_pizza.inputs.meal'), rdflib.term.URIRef('failed_ontology.make.outputs.rice'), [rdflib.term.URIRef('http://www.example.org/Pizza')], [rdflib.term.URIRef('http://www.example.org/Rice')])], 'distinct_units': {}}


## Downstream ontological hint is less specific

This should work fine...

In [34]:
wf = pwf.Workflow("relaxed_ontology")
wf.make = prepare_rice()
try:
    wf.eat = eat(wf.make)
except ChannelConnectionError as e:
    print(e)

The upstream channel /relaxed_ontology/make.rice cannot connect to the downstream channel /relaxed_ontology/eat.meal because the upstream type hint (typing.Annotated[__main__.Meal, ('uri', rdflib.term.URIRef('http://www.example.org/Rice'))]) and downstream type hint (typing.Annotated[__main__.Meal, ('uri', rdflib.term.URIRef('http://www.example.org/Meal'))]) produce a non-empty ontological validation report:
{'missing_triples': [], 'incompatible_connections': [(rdflib.term.URIRef('relaxed_ontology.eat.inputs.meal'), rdflib.term.URIRef('relaxed_ontology.make.outputs.rice'), [rdflib.term.URIRef('http://www.example.org/Meal')], [rdflib.term.URIRef('http://www.example.org/Rice')])], 'distinct_units': {}}


But! We forgot something! This form of failure is known from the `semantikon` notebook whence these demonstration workflow spring: we never informed the ontology that "rice" is a subclass of "meal"!

We let the ontology know this by adding the corresponding triple to our `rdflib.Graph`. In `pyiron_workflow` we can manage this by pre-populating a `knowledge: rdflib.Graph` property on the graph root (i.e. top-most object) as follows:

In [35]:
wf = pwf.Workflow("relaxed_ontology")

wf.knowledge = rdflib.Graph()
wf.knowledge.add((EX.Rice, rdflib.RDFS.subClassOf, EX.Meal))

wf.make = prepare_rice()
wf.eat = eat(wf.make)
wf()

{'eat__verdict': 'Yummy Meal meal'}

# Ontological triples

Alright, for our simple pizza example things are working beautifully. Let's try it with the clothes example. Note that the `derived_from` annotation does not cause the `uri` annotation to be inherited, so even when we derive our output from our input, we re-state the URI.

In [36]:
EX = rdflib.Namespace("http://www.example.org/")

class Clothes:
    pass

@pwf.as_function_node
def wash(clothes: u(Clothes, uri=EX.Clothes)) -> u(
    Clothes,
    uri=EX.Clothes,
    triples=(EX.hasProperty, EX.cleaned),
    derived_from="inputs.clothes"
):
    ...
    return clothes

@pwf.as_function_node
def dye(clothes: u(Clothes, uri=EX.Clothes), color="blue") -> u(
    Clothes,
    uri=EX.Clothes,
    triples=(EX.hasProperty, EX.color),
    derived_from="inputs.clothes",
):
    ...
    return clothes

@pwf.as_function_node
def sell(
    clothes: u(
        Clothes,
        uri=EX.Clothes,
        restrictions=(
            ((rdflib.OWL.onProperty, EX.hasProperty), (rdflib.OWL.someValuesFrom, EX.cleaned)),
            ((rdflib.OWL.onProperty, EX.hasProperty), (rdflib.OWL.someValuesFrom, EX.color)),
        )
    )
) -> int:
    price = 10
    return price

## Now with `restrictions`

In the base case, everything works fine. The restrictions are correctly parsed.

Note that unlike the `semantikon` notebook, here we had to make sure that all the node inputs are also `u` annotated (even if it's just to trivially link the type to its ontology counterpart). This is because type checking only occurs in `pyiron_workflow` when _both_ sides of the connection are typed! We follow this rule for both standard data types and ontological types.

In [37]:
my_correct_wf = pwf.Workflow("my_correct_workflow")
my_correct_wf.dyed_clothes = dye(Clothes())
my_correct_wf.washed_clothes = wash(my_correct_wf.dyed_clothes)
my_correct_wf.money = sell(my_correct_wf.washed_clothes)
my_correct_wf()

{'money__price': 10}

## As a macro

This also works fine! Be careful though, here we've only demonstrated that it _can_ work for macros, and have not yet guaranteed it works for _all_ macros.

In [38]:
@pwf.as_macro_node
def my_correct_macro(self, clothes: Clothes):
    self.dyed_clothes = dye(clothes)
    self.washed_clothes = wash(self.dyed_clothes)
    self.money = sell(self.washed_clothes)
    return self.money

correct_m = my_correct_macro(Clothes())
correct_m()

{'money': 10}

## Trivial failure

If we skip a step, our `sell` `restrictions` are not fulfilled, and we sensibly fail.

In [39]:
my_wrong_wf = pwf.Workflow("my_wrong_workflow")
my_wrong_wf.washed_clothes = wash(Clothes())
try:
    my_wrong_wf.money = sell(my_wrong_wf.washed_clothes)
except ChannelConnectionError as e:
    print(e)

The upstream channel /my_wrong_workflow/washed_clothes.clothes cannot connect to the downstream channel /my_wrong_workflow/sell.clothes because the upstream type hint (typing.Annotated[__main__.Clothes, ('uri', rdflib.term.URIRef('http://www.example.org/Clothes'), 'triples', (rdflib.term.URIRef('http://www.example.org/hasProperty'), rdflib.term.URIRef('http://www.example.org/cleaned')), 'derived_from', 'inputs.clothes')]) and downstream type hint (typing.Annotated[__main__.Clothes, ('uri', rdflib.term.URIRef('http://www.example.org/Clothes'), 'restrictions', (((rdflib.term.URIRef('http://www.w3.org/2002/07/owl#onProperty'), rdflib.term.URIRef('http://www.example.org/hasProperty')), (rdflib.term.URIRef('http://www.w3.org/2002/07/owl#someValuesFrom'), rdflib.term.URIRef('http://www.example.org/cleaned'))), ((rdflib.term.URIRef('http://www.w3.org/2002/07/owl#onProperty'), rdflib.term.URIRef('http://www.example.org/hasProperty')), (rdflib.term.URIRef('http://www.w3.org/2002/07/owl#someValu

## Macro failure

When we wrap the failing code as a macro, we don't fail until we try to instantiate that macro -- that is the first time the recipe code is evaluated and ontologically evaluated, at which point we fail at the connection formation just like in the workflow example.

In the future, if we move to `pyiron_workflow` decorators first producing (and validating) `flowrep` recipes and _then_ using these to create `pyiron_workflow` node classes, we'd be able to nicely fail at the macro definition time instead!

In [40]:
@pwf.as_macro_node
def my_wrong_macro(self, clothes: Clothes):
    self.washed_clothes = wash(clothes)
    self.money = sell(self.washed_clothes)
    return self.money

try:
    my_wrong_macro()
except ChannelConnectionError as e:
    print(e)

The upstream channel /my_wrong_macro/washed_clothes.clothes cannot connect to the downstream channel /my_wrong_macro/sell.clothes because the upstream type hint (typing.Annotated[__main__.Clothes, ('uri', rdflib.term.URIRef('http://www.example.org/Clothes'), 'triples', (rdflib.term.URIRef('http://www.example.org/hasProperty'), rdflib.term.URIRef('http://www.example.org/cleaned')), 'derived_from', 'inputs.clothes')]) and downstream type hint (typing.Annotated[__main__.Clothes, ('uri', rdflib.term.URIRef('http://www.example.org/Clothes'), 'restrictions', (((rdflib.term.URIRef('http://www.w3.org/2002/07/owl#onProperty'), rdflib.term.URIRef('http://www.example.org/hasProperty')), (rdflib.term.URIRef('http://www.w3.org/2002/07/owl#someValuesFrom'), rdflib.term.URIRef('http://www.example.org/cleaned'))), ((rdflib.term.URIRef('http://www.w3.org/2002/07/owl#onProperty'), rdflib.term.URIRef('http://www.example.org/hasProperty')), (rdflib.term.URIRef('http://www.w3.org/2002/07/owl#someValuesFrom

## Complex failure

Now let's be a little sneaky -- as usual, our "dye" node will add "color" to the clothes, but let's leverage our ontological power to _remove_ the "clean" state!

In [41]:
@pwf.as_function_node
def dye_with_cancel(clothes: Clothes, color="blue") -> u(
    Clothes,
    uri=EX.Clothes,
    derived_from="inputs.clothes",
    triples=(EX.hasProperty, EX.color),
    cancel=(EX.hasProperty, EX.cleaned)
):
    return clothes

We fail, as expected. The error messages for failed ontology validations are still extremely opaque, but we can see that the upstream node `'cancel'`s the `.../cleaned` property, while the downstream type hint still requires `hasProperty` `cleaned`.

In [42]:
my_wf_with_cancellation = pwf.Workflow("my_wf_with_cancellation")
my_wf_with_cancellation.washed_clothes = wash(Clothes())
my_wf_with_cancellation.dyed_clothes = dye_with_cancel(my_wf_with_cancellation.washed_clothes)
try:
    my_wf_with_cancellation.money = sell(my_wf_with_cancellation.dyed_clothes)
except ChannelConnectionError as e:
    print(e)

The upstream channel /my_wf_with_cancellation/dyed_clothes.clothes cannot connect to the downstream channel /my_wf_with_cancellation/sell.clothes because the upstream type hint (typing.Annotated[__main__.Clothes, ('uri', rdflib.term.URIRef('http://www.example.org/Clothes'), 'triples', (rdflib.term.URIRef('http://www.example.org/hasProperty'), rdflib.term.URIRef('http://www.example.org/color')), 'derived_from', 'inputs.clothes', 'extra', {'cancel': (rdflib.term.URIRef('http://www.example.org/hasProperty'), rdflib.term.URIRef('http://www.example.org/cleaned'))})]) and downstream type hint (typing.Annotated[__main__.Clothes, ('uri', rdflib.term.URIRef('http://www.example.org/Clothes'), 'restrictions', (((rdflib.term.URIRef('http://www.w3.org/2002/07/owl#onProperty'), rdflib.term.URIRef('http://www.example.org/hasProperty')), (rdflib.term.URIRef('http://www.w3.org/2002/07/owl#someValuesFrom'), rdflib.term.URIRef('http://www.example.org/cleaned'))), ((rdflib.term.URIRef('http://www.w3.org/2

# It's alpha

So far this has worked splendidly... for the particular test cases we're looking at. This is an alpha-feature and we neither support all possible `pyiron_workflow` node types, nor have we searched for and tested possible failing edge-cases among the supported node types. Thus, there is a safety valve. To turn off ontological validation, just go to the root-most object and set `._validate_ontologies = False`:

In [43]:
my_silenced_ontology = pwf.Workflow("my_silenced_ontology")
my_silenced_ontology._validate_ontologies = False
my_silenced_ontology.washed_clothes = wash(Clothes())
my_silenced_ontology.dyed_clothes = dye_with_cancel(my_silenced_ontology.washed_clothes)
my_silenced_ontology.money = sell(my_silenced_ontology.dyed_clothes)
my_silenced_ontology()

{'money__price': 10}

# Node suggestions

One of the advantages of graph-based workflows with hinted IO channels is facilitating guided workflow creation. Given a hinted channel instance in the context of some workflow, we can ask for suggestions of other channels with which to form a connection in the same, sibling graph context:

In [44]:
wf = pwf.Workflow("ontoflow")
wf.make = prepare_pizza()
wf.eat = eat_pizza()
suggestions = suggest.suggest_connections(wf.eat.inputs.meal)
for (node, channel) in suggestions:
    print(node.full_label, channel.label)

/ontoflow/make pizza


Similarly, given a corpus of node classes, we can ask for which nodes have at least one commensurate input/output with which our channel might connect. After adding such a node to our graph, we can leverage the connection suggester to see which channel(s) are appropriate.

In [45]:
suggest.suggest_nodes(wf.eat.inputs.meal, pwf.std.UserInput, prepare_pizza, wash)

[__main__.prepare_pizza]

## Suggestion limitations

When searching for new upstream nodes to add, the current implementation only looks at the immediate node, and not possible trees of upstream nodes. Returning to our clothes example, we can see that there is no _single_ suggestion for the `sell` node, because it requires clothes that are both dyed _and_ coloured, but our other nodes only provide one of these at a time!

In [46]:
clothing_nodes = wash, dye, dye_with_cancel, sell

wf = pwf.Workflow("working_backwards")
wf.money = sell()
suggest.suggest_nodes(wf.money, *clothing_nodes)

[]

Of course working backwards a single step still works fine for lots of nodes, e.g. for `dye` we will take _anything_ that gives us clothes!

In [47]:
wf = pwf.Workflow("single_step_back")
wf.dyed_clothes = dye()
suggest.suggest_nodes(wf.dyed_clothes, *clothing_nodes)

[__main__.wash, __main__.dye, __main__.dye_with_cancel]

And when we look _downstream_ we have the advantage of knowing the entire upstream graph concretely, so there we are able to see options for fulfilling these more complex demands.

In [48]:
wf = pwf.Workflow("downstream")
wf.dyed_clothes = dye(Clothes())
wf.washed_clothes = wash(wf.dyed_clothes)
suggestions = suggest.suggest_nodes(wf.washed_clothes, *clothing_nodes)
assert(sell in suggestions)
print(suggestions)

[<class '__main__.wash'>, <class '__main__.dye'>, <class '__main__.dye_with_cancel'>, <class '__main__.sell'>]


Note that inter-node connections do _not_ cause restrictions to propagate. Nicely, that means that if we have a node guarnateeing our immediate demands, we can get suggestions for it, even if it would itself introduce new demands. See below, where the middle node `GivesAndTakes` promises to fulfill the requirements of `TakesDownstream` even though it has its own needs:

In [49]:
@pwf.as_function_node
def GivesUpstreamNeed(x) -> u(str, uri=EX.Data, triples=(EX.has, EX.Upstream)):
    return str(x)

@pwf.as_function_node
def GivesAndTakes(
    y: u(
        str,
        uri=EX.Data,
        restrictions=(
            (rdflib.OWL.onProperty, EX.has),
            (rdflib.OWL.someValuesFrom, EX.Upstream),
        ),
    ),
) -> u(
    str,
    uri=EX.Data,
    derived_from="inputs.y",
    triples=(EX.has, EX.Downstream)
):
    return y

@pwf.as_function_node
def TakesDownstream(
    z: u(
        str,
        uri=EX.Data,
        restrictions=(
            (rdflib.OWL.onProperty, EX.has),
            (rdflib.OWL.someValuesFrom, EX.Downstream),
        ),
    ),
) -> str:
    return z

wf = pwf.Workflow("derived_restrictions")
wf.up = GivesUpstreamNeed()
wf.middle = GivesAndTakes()
wf.down = TakesDownstream()

suggest.suggest_connections(wf.middle.outputs.y)

[(<__main__.TakesDownstream at 0x132987350>,
  <pyiron_workflow.channels.InputData at 0x132986420>)]

## Complex workflows

Ontological validation is still a new feature, and you may find an edge case we haven't found and tested yet. However, we can see below that even complex graphs including macros, dataclass nodes, and for-loops are able to validate and run:

In [50]:
EX = rdflib.Namespace("http://www.example.org/")

@pwf.as_dataclass_node(uri=EX.Jar)  # Can pass u-kwargs to decorate the returned dataclass
class Jar:
    threading: str = "clockwise"
    contents: u(str, uri=EX.Contents) = "jam"

@pwf.as_function_node
def ItsStuck(
    jar: u(Jar.dataclass, uri=EX.Jar)
) -> u(
        Jar.dataclass,
        uri=EX.Jar,
        derived_from="inputs.jar",
        triples=(EX.lidState, EX.stuck)
):
    return jar

@pwf.as_function_node
def OpenStuckJar(
    jar: u(
        Jar.dataclass,
        uri=EX.Jar,
        restrictions=(
            (rdflib.OWL.onProperty, EX.lidState),
            (rdflib.OWL.someValuesFrom, EX.stuck),
        ),
    ),
) -> u(str, uri=EX.Contents):
    contents = jar.contents
    return contents

@pwf.as_function_node
def MakeSandwich(made_with: u(str, uri=EX.Contents)) -> u(str, uri=EX.Sandwich):
    sandwich = f"{made_with} sandwich"
    return sandwich

@pwf.as_macro_node
def LunchTime(
    self, contents: u(str, uri=EX.Contents)
) -> u(str, uri=EX.Sandwich, triples=(EX.madeWith, "inputs.contents")):
    self.jar = Jar(contents=contents)
    self.stuck_jar = ItsStuck(self.jar)
    self.open_jar = OpenStuckJar(self.stuck_jar)
    self.lunch = MakeSandwich(self.open_jar)
    return self.lunch



wf = pwf.Workflow("lunch_for_three")
wf.platter = pwf.for_node(
    body_node_class=LunchTime,
    iter_on="contents",
    contents=["jam", "honey", "butter"],
    output_as_dataframe=False,
)
wf()

{'platter__contents': ['jam', 'honey', 'butter'],
 'platter__lunch': ['jam sandwich', 'honey sandwich', 'butter sandwich']}

# Units

`semantikon` annotations also allow us to specify physical units. When present, these are included in the ontological validation just like the other ontological terms.

As such, we have no problem making same-unit connections:

In [51]:
@pwf.as_function_node
def Distance(x: u(float, units="meter")) -> u(float, derived_from="inputs.x"):
    return x

@pwf.as_function_node
def Speed(
        dx: u(float, units="meter"), dt: u(float, units="second")
) -> u(float, units="meter/second"):
    s = dx/dt
    return s

wf = pwf.Workflow("speedometer")
wf.dx = Distance(100)
wf.speed = Speed(dx=wf.dx)

With incompatible units, we get an exception at connection time, just like with other ontological failures:

In [52]:
@pwf.as_function_node
def NanoTime(t: u(float, units="nanosecond")) -> u(float, units="nanosecond"):
    return t

wf.dt = NanoTime(10)
try:
    wf.speed.inputs.dt = wf.dt
except ChannelConnectionError as e:
    print(e)
    wf.remove_child(wf.dt)

The upstream channel /speedometer/dt.t cannot connect to the downstream channel /speedometer/speed.dt because the upstream type hint (typing.Annotated[float, ('units', 'nanosecond')]) and downstream type hint (typing.Annotated[float, ('units', 'second')]) produce a non-empty ontological validation report:
{'missing_triples': [], 'incompatible_connections': [], 'distinct_units': {rdflib.term.URIRef('speedometer.dt.outputs.t.value'): [rdflib.term.URIRef('http://qudt.org/vocab/unit/NanoSEC'), rdflib.term.URIRef('http://qudt.org/vocab/unit/SEC')]}}


With correct units, it works fine

In [53]:
@pwf.as_function_node
def Time(t: u(float, units="second")) -> u(float, units="second"):
    return t

wf.dt = Time(10)
wf.speed.inputs.dt = wf.dt
wf()

{'speed__s': 10.0}

Note that units are not inherited when `derived_from=` is used in the annotation; this is to easily allow for unit-converting nodes.

# Known Issues

- This implementation naively creates a circular dependence: the `channels` module needs the `knowledge` module to evaluate the ontological validity of new connections, but the `knowledge` module relies on `workflow` and `nodes.composite` to parse graphs, and these in turn depend on `channels`. For now, we avoid dealing with this by importing `knowledge` locally in `channels` when it's time to use the ontology.
- There are strings everywhere. The ontological features rely heavily on dictionaries, which are tough to type check and rely on string-based key access. E.g., when we want to see if the ontological validation raised any errors, we need to manually check on two dictionary entries by name. This is fragile.
- It is inefficient. At every new ontologically-hinted connection, we reconstruct the entire recipe dictionary before positing the new connection and checking its validity. I'm not sure we'll get around validation operating on the entire graph, but we should adjust `pyiron_workflow` to store more recipe information at the class level where it is statically known (macros, function nodes, etc)
- This is not fully edge-case tested. This works for the cases we show here and in the tests, but work in the underlying ontological validation in `semantikon` is ongoing, and there may be `pyiron_workflow`-specific syntax that is not yet playing well with `semantikon`. If you run into a problem, please raise a GitHub issue!
- Upstream suggestions are limited to a _single_ suggestion, we don't do any tree construction to create upstream subgraphs that leverage trees of nodes in order to fulfill ontological demands. We _could_, but a brute-force attack would scale horribly with the node corpus.
- Overhead: importing `semantikon.ontology` takes the better part of a second. We delay the import until the last moment, so this only impacts graphs where both ends of a connection are annotated, but there the import time is slow enough to be noticed on human scales.

